In [1]:
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import asyncio

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select


# Descargamos el json con los datos que queremos:

In [2]:
import requests

url = 'https://datos.madrid.es/egob/catalogo/209548-491-censo-locales-historico.json'  

response = requests.get(url)

if response.status_code == 200:
    
    data = response.json()
    df = pd.DataFrame(data)

else:
    print("No se pudo descargar el archivo JSON.")


In [3]:
data=df.copy()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151079 entries, 0 to 151078
Data columns (total 48 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_local                   151079 non-null  int64  
 1   id_distrito_local          151079 non-null  int64  
 2   desc_distrito_local        151079 non-null  object 
 3   id_barrio_local            151079 non-null  int64  
 4   desc_barrio_local          151079 non-null  object 
 5   cod_barrio_local           151079 non-null  int64  
 6   id_seccion_censal_local    151079 non-null  int64  
 7   desc_seccion_censal_local  151079 non-null  int64  
 8   coordenada_x_local         151079 non-null  float64
 9   coordenada_y_local         151079 non-null  float64
 10  id_tipo_acceso_local       151079 non-null  int64  
 11  desc_tipo_acceso_local     151079 non-null  object 
 12  id_situacion_local         151079 non-null  int64  
 13  desc_situacion_local       15

In [5]:
data.head()

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,cod_postal,fx_carga,fx_datos_ini,fx_datos_fin,coordenada_x_agrupacion,coordenada_y_agrupacion,hora_apertura1,hora_cierre1,hora_apertura2,hora_cierre2
0,10000004,1,CENTRO,105,UNIVERSIDAD,5,1106,106,439945.60,4475591.53,...,28015,2023-10-08T07:02:06,2023-10-01,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN
1,10000116,1,CENTRO,105,UNIVERSIDAD,5,1112,112,0.00,0.00,...,28015,2023-10-08T07:02:06,2023-10-01,2023-10-01,439852.60,4474996.53,NaN,NaN,NaN,NaN
2,10000150,1,CENTRO,106,SOL,6,1121,121,440044.59,4474063.53,...,28012,2023-10-08T07:02:06,2023-10-01,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN
3,10000224,1,CENTRO,102,EMBAJADORES,2,1035,35,0.00,0.00,...,28012,2023-10-08T07:02:06,2023-10-01,2023-10-01,440295.59,4473246.53,NaN,NaN,NaN,NaN
4,10000264,1,CENTRO,106,SOL,6,1124,124,440226.59,4474520.53,...,28013,2023-10-08T07:02:06,2023-10-01,2023-10-01,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Comnezamos eliminando las columnas inecesarias

data = data.drop(columns=['hora_cierre2','hora_apertura2','fx_carga','cal_acceso','num_acceso','nom_acceso','id_seccion_censal_local','desc_seccion_censal_local','coordenada_x_agrupacion','coordenada_y_agrupacion','hora_cierre1','hora_apertura1'])

In [7]:
#data = data.drop(columns=['id_tipo_acceso_local','desc_tipo_acceso_local','id_agrupacion'])
#data = data.drop(columns=['id_vial_edificio','clase_vial_edificio'])
#data = data.drop(columns=['id_situacion_local','desc_situacion_local','id_clase_ndp_acceso','nombre_agrupacion','id_tipo_agrup','desc_tipo_agrup','id_planta_agrupado','id_local_agrupado','fx_datos_ini','fx_datos_fin'])

In [8]:
data = data.drop(columns=['id_situacion_local','desc_situacion_local','id_clase_ndp_acceso','nombre_agrupacion','id_tipo_agrup','desc_tipo_agrup','id_planta_agrupado','id_local_agrupado','fx_datos_ini','fx_datos_fin'])

In [9]:
data.columns

Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_vial_edificio', 'clase_vial_edificio',
       'desc_vial_edificio', 'id_ndp_edificio', 'id_clase_ndp_edificio',
       'nom_edificio', 'num_edificio', 'cal_edificio', 'secuencial_local_PC',
       'id_vial_acceso', 'clase_vial_acceso', 'desc_vial_acceso',
       'id_ndp_acceso', 'id_agrupacion', 'rotulo', 'cod_postal'],
      dtype='object')

In [10]:
data.reset_index()

,index,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,...,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_agrupacion,rotulo,cod_postal
0,0,10000004,1,CENTRO,105,UNIVERSIDAD,5,439945.60,4475591.53,1,...,31,,10,4700,CALLE,ACUERDO ...,11006452,-1,CAMDEN COFFEE ROASTERS,28015
1,1,10000116,1,CENTRO,105,UNIVERSIDAD,5,0.00,0.00,0,...,1,,0,524500,PLAZA,MOSTENSES ...,11006826,99000061,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22,28015
2,2,10000150,1,CENTRO,106,SOL,6,440044.59,4474063.53,1,...,1,,10,498000,PLAZA,MAYOR ...,11007683,-1,CAFE LOS ARCOS BAR,28012
3,3,10000224,1,CENTRO,102,EMBAJADORES,2,0.00,0.00,0,...,41,,0,262000,CALLE,EMBAJADORES ...,11002143,99000057,PAIS DE LOS CEVICHES L01/ANTIGUO PUESTO 70,28012
4,4,10000264,1,CENTRO,106,SOL,6,440226.59,4474520.53,1,...,4,,10,505100,CALLE,MESONERO ROMANOS ...,11007866,-1,VALDEMESO,28013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151074,151074,300021996,13,PUENTE DE VALLECAS,1306,NUMANCIA,6,0.00,0.00,12,...,120,,10,88940501,CAMINO,VALDERRIBAS ...,31042710,-1,SIN ACTIVIDAD,28038
151075,151075,300021997,13,PUENTE DE VALLECAS,1301,ENTREVIAS,1,0.00,0.00,12,...,6,,10,45800,CALLE,ANDEVALO ...,31022765,-1,SIN ACTIVIDAD,28053
151076,151076,300021999,20,SAN BLAS-CANILLEJAS,2005,ROSAS,5,0.00,0.00,12,...,1,,10,2355,CALLE,CARLOS II ...,31028964,-1,SIN ACTIVIDAD,28022
151077,151077,300022007,13,PUENTE DE VALLECAS,1303,PALOMERAS BAJAS,3,0.00,0.00,12,...,8,,10,57800,CALLE,ANTONIO FOLGUERAS ...,11095000,-1,SERVICIO DE COMEDOR COLEGIO MATER CLEMENTISSMA,28018


In [11]:
data=data.drop(columns=[])

In [12]:
import numpy as np
nueva_columna = np.arange(1, len(df) + 1)
data.id_local=nueva_columna

In [13]:
data.columns

Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_vial_edificio', 'clase_vial_edificio',
       'desc_vial_edificio', 'id_ndp_edificio', 'id_clase_ndp_edificio',
       'nom_edificio', 'num_edificio', 'cal_edificio', 'secuencial_local_PC',
       'id_vial_acceso', 'clase_vial_acceso', 'desc_vial_acceso',
       'id_ndp_acceso', 'id_agrupacion', 'rotulo', 'cod_postal'],
      dtype='object')

In [14]:
data=data.drop(columns='desc_vial_edificio')

In [15]:
mascara = data['rotulo'] != 'SIN ACTIVIDAD'

# Utiliza la máscara para seleccionar las filas que no coinciden con "SIN ACTIVIDAD"
data = data[mascara]

In [16]:
data.head()

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,desc_tipo_acceso_local,...,num_edificio,cal_edificio,secuencial_local_PC,id_vial_acceso,clase_vial_acceso,desc_vial_acceso,id_ndp_acceso,id_agrupacion,rotulo,cod_postal
0,1,1,CENTRO,105,UNIVERSIDAD,5,439945.60,4475591.53,1,Puerta Calle,...,31,,10,4700,CALLE,ACUERDO ...,11006452,-1,CAMDEN COFFEE ROASTERS,28015
1,2,1,CENTRO,105,UNIVERSIDAD,5,0.00,0.00,0,Agrupado,...,1,,0,524500,PLAZA,MOSTENSES ...,11006826,99000061,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22,28015
2,3,1,CENTRO,106,SOL,6,440044.59,4474063.53,1,Puerta Calle,...,1,,10,498000,PLAZA,MAYOR ...,11007683,-1,CAFE LOS ARCOS BAR,28012
3,4,1,CENTRO,102,EMBAJADORES,2,0.00,0.00,0,Agrupado,...,41,,0,262000,CALLE,EMBAJADORES ...,11002143,99000057,PAIS DE LOS CEVICHES L01/ANTIGUO PUESTO 70,28012
4,5,1,CENTRO,106,SOL,6,440226.59,4474520.53,1,Puerta Calle,...,4,,10,505100,CALLE,MESONERO ROMANOS ...,11007866,-1,VALDEMESO,28013


In [17]:
localizacion = ['id_local','id_distrito_local','desc_distrito_local','id_barrio_local', 'desc_barrio_local', 'cod_barrio_local','coordenada_x_local', 'coordenada_y_local']
loc = pd.DataFrame(data, columns=localizacion)
loc

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,coordenada_x_local,coordenada_y_local
0,1,1,CENTRO,105,UNIVERSIDAD,5,439945.60,4475591.53
1,2,1,CENTRO,105,UNIVERSIDAD,5,0.00,0.00
2,3,1,CENTRO,106,SOL,6,440044.59,4474063.53
3,4,1,CENTRO,102,EMBAJADORES,2,0.00,0.00
4,5,1,CENTRO,106,SOL,6,440226.59,4474520.53
...,...,...,...,...,...,...,...,...
151066,151067,10,LATINA,1004,ALUCHE,4,0.00,0.00
151068,151069,9,MONCLOA-ARAVACA,903,CIUDAD UNIVERSITARIA,3,0.00,0.00
151069,151070,8,FUENCARRAL-EL PARDO,805,LA PAZ,5,0.00,0.00
151070,151071,12,USERA,1205,MOSCARDO,5,0.00,0.00


In [18]:
coor=['id_local','coordenada_x_local','coordenada_y_local']
coordenadas=pd.DataFrame(loc, columns=coor)
coordenadas

,id_local,coordenada_x_local,coordenada_y_local
0,1,439945.60,4475591.53
1,2,0.00,0.00
2,3,440044.59,4474063.53
3,4,0.00,0.00
4,5,440226.59,4474520.53
...,...,...,...
151066,151067,0.00,0.00
151068,151069,0.00,0.00
151069,151070,0.00,0.00
151070,151071,0.00,0.00


In [19]:
mascara = (coordenadas['coordenada_x_local'] == 0) & (coordenadas['coordenada_y_local'] == 0)

# Utiliza la máscara para eliminar las filas correspondientes, las que son 0
coordenadas = coordenadas[~mascara]

In [20]:
coordenadas = coordenadas.astype(int)

In [21]:
coordenadas.describe().T

,count,mean,std,min,25%,50%,75%,max
id_local,102415.0,6.136123e+04,39916.833989,1.0,27168.5,57996.0,89341.5,151023.0
coordenada_x_local,102415.0,4.417619e+05,3386.918902,429079.0,439801.0,441089.0,443790.0,454722.0
coordenada_y_local,102415.0,4.474627e+06,3863.770085,4463368.0,4471766.0,4474873.0,4477144.0,4495496.0


In [22]:
data.columns

Index(['id_local', 'id_distrito_local', 'desc_distrito_local',
       'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local',
       'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local',
       'desc_tipo_acceso_local', 'id_vial_edificio', 'clase_vial_edificio',
       'id_ndp_edificio', 'id_clase_ndp_edificio', 'nom_edificio',
       'num_edificio', 'cal_edificio', 'secuencial_local_PC', 'id_vial_acceso',
       'clase_vial_acceso', 'desc_vial_acceso', 'id_ndp_acceso',
       'id_agrupacion', 'rotulo', 'cod_postal'],
      dtype='object')

In [23]:
rest=['id_local','rotulo']
restaurantes=pd.DataFrame(data, columns=rest)
restaurantes

,id_local,rotulo
0,1,CAMDEN COFFEE ROASTERS
1,2,PERFUMERIA DROGUERIA RICARDO / LOCAL 17 AL 22
2,3,CAFE LOS ARCOS BAR
3,4,PAIS DE LOS CEVICHES L01/ANTIGUO PUESTO 70
4,5,VALDEMESO
...,...,...
151066,151067,CAFETERIA CENTRO DEPORTIVO SAGE DOS MIL
151068,151069,"CAFETERIA ETSI MONTES, FORESTAL Y DEL MEDIO NA..."
151069,151070,ESCUELA INFANTIL ISABEL ZENDAL
151070,151071,CAFETERIA CLUB DEPORTIVO COLONIA MOSCARDO


In [27]:
import sys
sys.path.append("..")
from src.funciones import *

optim(restaurantes)
restaurantes.to_csv('../data/restaurantes.csv')